In [40]:
import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import tensorflow as tf

In [41]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import boto3
from boto3.s3.transfer import TransferConfig

In [42]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [43]:
def LoadFromS3(i):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Load for File",i)
    embeddings = {}
    session = boto3.session.Session(region_name='us-east-1')
    s3client = session.client('s3')
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open(i, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("File Loaded in:", time.time()-now)
    
    return embeddings

In [44]:
def SaveToS3(file, name):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Write for File", name)
    file = file.to_json()
    file.save(f'{name}.h5')
    session = boto3.session.Session(region_name='us-east-1')
    s3 = boto3.resource('s3')
    object = s3.Object('w210-mimic', name)
    object.put(Body=file)

    print("File Saved in:", time.time()-now)
    
    return

In [45]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
opt = keras.optimizers.Adam(lr=0.0001)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

In [24]:
# session = boto3.session.Session(region_name='us-east-1')
# s3client = session.client('s3')
# response = s3client.get_object(Bucket='w210-mimic', Key="brent_chapter_2_embeddings6")
# body_string = response['Body'].read()

In [26]:
# model = tf.keras.models.model_from_json(body_string)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [46]:
# Saving the Files Locally:

model_json = model.to_json()

with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5")


In [48]:
EPOCHS = 5

In [ ]:
for epoch in range(EPOCHS):
    for i in ['embeddings6', 'embeddings5', 'embeddings4', 'embeddings3', 'embeddings2', 'embeddings1']:
        data = LoadFromS3(i)
        
        data = np.array(list(data.items()))
        HADM_ID_LIST = data[:,0]
        data = data[:,1]

        data = [np.float16(np.concatenate(i[:30])) for i in data]

        embeddings_padded = []
        
        print("Padding Embeddings:")
        for j in data:
            pad_len = 3840 - len(j)
            if pad_len:
                embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
            else:
                embeddings_padded.append(j)
        
        del(data)
        embeddings_padded = np.float16(embeddings_padded)
        embeddings_padded = np.array(embeddings_padded, dtype=np.float16)

        CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/chapter_label.csv', sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

        EMBEDDINGS_CHAPTER_PIVOT_DF = pd.DataFrame(data=list(zip(HADM_ID_LIST, embeddings_padded)), columns=['HADM_ID', 'embeddings']).set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left')
        EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.fillna(0)
        EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.iloc[:,1:]
        EMBEDDINGS_CHAPTER_PIVOT_NP = np.array(EMBEDDINGS_CHAPTER_PIVOT_DF)
        del(EMBEDDINGS_CHAPTER_PIVOT_DF)

        print("Training Epoch:", epoch, "Embeddings:",i)
        history = model.fit(embeddings_padded, EMBEDDINGS_CHAPTER_PIVOT_NP, epochs=1, batch_size=256, verbose=1, validation_split=0.1)
        model.save(f"model_{epoch}_{i}.hdr")


Starting Load for File embeddings6
File Loaded in: 731.976432800293
Padding Embeddings:
Training Epoch: 0 Embeddings: embeddings6


Train on 2453 samples, validate on 273 samples
Epoch 1/1
2453/2453 [==============================] - 278s 113ms/step - loss: 0.6857 - get_f1: 0.5315 - val_loss: 0.6698 - val_get_f1: 0.5625
Starting Load for File embeddings5
File Loaded in: 3009.5957169532776
Padding Embeddings:
Training Epoch: 0 Embeddings: embeddings5
Train on 9000 samples, validate on 1000 samples
Epoch 1/1
9000/9000 [==============================] - 1010s 112ms/step - loss: 0.6503 - get_f1: 0.6043 - val_loss: 0.6404 - val_get_f1: 0.6076
Starting Load for File embeddings4
File Loaded in: 2601.6787514686584
Padding Embeddings:
Training Epoch: 0 Embeddings: embeddings4
Train on 9000 samples, validate on 1000 samples
Epoch 1/1
9000/9000 [==============================] - 1017s 113ms/step - loss: 0.6513 - get_f1: 0.5628 - val_loss: 0.6517 - val_get_f1: 0.5483
Starting Load for File embeddi

# Upload Final Model & Weights to S3

In [52]:
2+2

4

# Cells Below are for Debugging & Legacy

In [51]:
2+2

4